In [1]:
import joblib
import pandas as pd
import unicodedata
import re
import pickle
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, mean_squared_error

In [2]:
pipeline_path = "/Users/fabiancordenod/code/fqbq69/BIMpredict-/pipeline/randomforestsolspipeline.pkl"
pipeline = joblib.load(pipeline_path)

In [3]:
# Charger le CSV
csv_path = "/Users/fabiancordenod/code/fqbq69/BIMpredict-/datatest/test21020/sols21020.csv"
df_test = pd.read_csv(csv_path, delimiter=';', header=1)

# Nettoyer les noms de colonnes comme à l'entraînement
def clean_col(col):
    col = ''.join(c for c in unicodedata.normalize('NFD', col) if unicodedata.category(c) != 'Mn')
    col = col.lower()
    col = re.sub(r"[ \-\(\)]", "_", col)
    col = re.sub(r"[^a-z0-9_]", "", col)
    col = re.sub(r"_+", "_", col)
    col = col.strip("_")
    return col

df_test.columns = [clean_col(c) for c in df_test.columns]

In [4]:
# Les targets à prédire (après nettoyage)
targets = [
    "011ec_lot",
    "012ec_ouvrage",
    "013ec_localisation",
    "014ec_mode_constructif"
]

# Colonnes explicatives attendues par le pipeline
features = pipeline.named_steps['preprocessor'].feature_names_in_

# S'assurer que toutes les colonnes sont présentes
for col in features:
    if col not in df_test.columns:
        df_test[col] = np.nan
X_test = df_test[features].copy()

In [5]:
# Remove duplicate columns if any (especially 'epaisseur')
X_test_unique = X_test.loc[:, ~X_test.columns.duplicated()]

# Reorder columns to match the order used during pipeline training
X_test_ordered = X_test_unique[features]
y_pred = pipeline.predict(X_test_ordered)

In [6]:
# Les targets à prédire (après nettoyage)
targets = [
    "011ec_lot",
    "012ec_ouvrage",
    "013ec_localisation",
    "014ec_mode_constructif"
]

# Mettre les prédictions dans un DataFrame
df_pred = pd.DataFrame(y_pred, columns=targets)

# Afficher les premières lignes
print(df_pred.head(50))

   011ec_lot 012ec_ouvrage 013ec_localisation 014ec_mode_constructif
0         GO       DALLAGE            COURANT         COULE EN PLACE
1         GO      PLANCHER            COURANT         COULE EN PLACE
2         GO      PLANCHER            COURANT         COULE EN PLACE
3         GO       DALLAGE            COURANT         COULE EN PLACE
4         GO       DALLAGE            COURANT         COULE EN PLACE
5         GO       DALLAGE            COURANT         COULE EN PLACE
6         GO      PLANCHER            COURANT         COULE EN PLACE
7         GO      PLANCHER            COURANT         COULE EN PLACE
8         GO      PLANCHER            COURANT         COULE EN PLACE
9         GO      PLANCHER            COURANT         COULE EN PLACE
10        GO      PLANCHER            COURANT         COULE EN PLACE
11        GO      PLANCHER            COURANT         COULE EN PLACE
12        GO      PLANCHER            COURANT         COULE EN PLACE
13        GO      PLANCHER        

In [7]:
df_pred.to_csv("resultats_predictionssols21020.csv", index=False)